In [1]:
import hashlib
name=''

In [2]:
class Teacher(object):
    def __init__(self,name,password):
        self.name=name
        self.password=encrypt_password(password)
        
class Student(object):
    def __init__(self,s_id,name,age,sex,tel):
        self.s_id=s_id
        self.name=name
        self.age=age
        self.sex=sex
        self.tel=tel

In [3]:
#prepare
def read_file(file_name):
    try:
        with open(file_name,'r',encoding='utf8') as file:
            content=file.read()
            return content
    except FileNotFoundError:
        print('文件未找到')
        
def write_json(file_name,data):
    with open(file_name,'w',encoding='utf8') as file:
        import json
        json.dump(data,file)
        
def read_json(file_name,default_data):
    try:
        with open(file_name,'r',encoding='utf8') as file:
            import json
            return json.load(file)
    except FileNotFoundError:
        return default_data

In [4]:
#student_manager
def add_student():
    x=read_json(name+'.json',{})
    if not x:
        students=[]
        n=0
    else:
        students=x['all_student']
        n=int(x['num'])
    while 1:
        student_name=input('请输入学生姓名：')
        student_age=input('请输入学生年龄：')
        student_sex=input('请输入学生性别：')
        student_tel=input('请输入学生电话号码：')
        n+=1
        s_id='stu'+str(n).zfill(4)
        s=Student(s_id,student_name,student_age,student_sex,student_tel)
        students.append(s.__dict__)
        data={'all_student':students,'num':len(students)}
        write_json(name+'.json',data)
        while 1:
            choice=input('添加成功！\n1.继续\n2.返回\n请选择（1-2）：')
            if choice=='1':
                break
            elif choice=='2':
                break
            else:
                print('输入有误！')
        if choice=='2':
            break

def show_student():
    key=value=''
    x=read_json(name+'.json',{})
    students=x.get('all_student',[])
    if not students:
        print('您还没有添加学生')
        return
    operator=input('1.查看所有学生\n2.根据姓名查找\n3.根据学号查找\n其它:返回\n请选择：')
    if operator=='1':
        pass
    elif operator=='2':
        value=input('请输入学生姓名：')
        key='name'
    elif operator=='3':
        value=input('请输入学生学号(格式：stu0001...)：')
        key='s_id'
    else:
        return
    students=filter(lambda s:s.get(key,'')==value,students)
    if not students:
        print('未找到该学生')
    for i in students:
        print('学号:{s_id},姓名:{name},性别:{sex},年龄:{age},☏:{tel}'.format(**i))

def delte_student():
    key=value=''
    x=read_json(name+'.json',{})
    all_students=x.get('all_student',[])
    if not all_students:
        print('您还没有添加学生')
        return
    operator=input('1.按姓名删\n2.按学号删\n其它：返回\n请选择：')
    if operator=='1':
        value=input('请输入学生姓名：')
        key='name'
    elif operator=='2':
        value=input('请输入学生学号(格式：stu0001...)：')
        key='s_id'
    else:
        return
    students=list(filter(lambda s:s.get(key,'')==value,all_students))
    if not students:
        print('未找到该学生')
        return
    elif len(students)==1:
        all_students.remove(students[0])
        print('删除成功！')
        x['all_student']=all_students
        write_json(name+'.json',x)
    else:
        for n,i in enumerate(students):
            print('{x} 学号:{s_id},姓名:{name},性别:{sex},年龄:{age},☏:{tel}'.format(x=n,**i))
        y=input('请输入需要删除的学生的标号(0-{}),其它则返回'.format(n))
        if y.isdigit():
            y=int(y)
            if 0<=y<=n:
                all_students.remove(students[int(y)])
                print('删除成功！')
                x['all_student']=all_students
                write_json(name+'.json',x)
            else:
                print('输入的内容不合法！')
                return
        else:
            print('输入的内容不合法！')
            return

def show_manager():
    while 1:
        print(read_file('students_page.txt')%name)
        operator=input('请选择(1-4)')
        if operator=='1':
            add_student()
        elif operator=='2':
            show_student()
        elif operator=='3':
            delte_student()
        elif operator=='4':
            break
        else:
            print('输入有误！')

In [5]:
#tools
def encrypt_password(password,x='makaiheng'):
    h=hashlib.sha256()
    h.update(password.encode('utf8'))
    h.update(x.encode('utf8'))
    return h.hexdigest()

In [6]:
#progress
def register():
    teachers=read_json('teachers.json',{})
    while 1:
        teacher_name=input('请输入账号(3-6位)：')
        if not 3<=len(teacher_name)<=6:
            print('账号不符合要求，请重新输入！')
        else:
            break
    if teacher_name in teachers:
        print('注册失败！该账号已经注册过！')
        return
    while 1:
        password=input('请输入密码(6-12位)：')
        if not 6<=len(password)<=12:
            print('密码不符合要求，请重新输入！')
        else:
            break
    t=Teacher(teacher_name,password)
    teachers[t.name]=t.password
    write_json('teachers.json',teachers)

def login():
    teachers=read_json('teachers.json',{})
    teacher_name=input('请输入老师账号：')
    if teacher_name not in teachers:
        print('登录失败！该账号没有注册！')
        return
    password=input('请输入密码：')
    if teachers[teacher_name]==encrypt_password(password):
        print('登录成功！')
        global name
        name=teacher_name
        show_manager()
    else:
        print('密码错误，登录失败！')

In [7]:
def start():
    while 1:
        print(read_file('welcome.txt'))
        operater=input('\n请选择(1-3):')
        if operater=='1':
            login()
        elif operater=='2':
            register()
        elif operater=='3':
            break
        else:
            print('输入有误！')
            
start()

    ** 欢迎来到学生管理系统 **

     ◆  1. 登    录
     ◆  2. 注    册
     ◆  3. 退    出
     

请选择(1-3):1
请输入老师账号：mkh
请输入密码：123456
登录成功！
    ❀❀欢迎mkh:
    
      ❤1.添加学生
      ❤2.查看学生
      ❤3.修改学生信息
      ❤4.删除学生
      ❤5.返回
请选择(1-5)2
1.查看所有学生
2.根据姓名查找
3.根据学号查找
其它:返回
请选择：1
学号:stu0001,姓名:zhangsan,性别:male,年龄:18,☏:110
学号:stu0002,姓名:lisi,性别:female,年龄:17,☏:120
学号:stu0003,姓名:zhangsan,性别:female,年龄:12,☏:123
学号:stu0004,姓名:lisi,性别:male,年龄:23,☏:999
学号:stu0005,姓名:wangwu,性别:male,年龄:19,☏:666
    ❀❀欢迎mkh:
    
      ❤1.添加学生
      ❤2.查看学生
      ❤3.修改学生信息
      ❤4.删除学生
      ❤5.返回
请选择(1-5)2
1.查看所有学生
2.根据姓名查找
3.根据学号查找
其它:返回
请选择：3
请输入学生学号(格式：stu0001...)：stu0003
学号:stu0003,姓名:zhangsan,性别:female,年龄:12,☏:123
    ❀❀欢迎mkh:
    
      ❤1.添加学生
      ❤2.查看学生
      ❤3.修改学生信息
      ❤4.删除学生
      ❤5.返回
请选择(1-5)4
1.按姓名删
2.按学号删
其它：返回
请选择：1
请输入学生姓名：zhangsan
0 学号:stu0001,姓名:zhangsan,性别:male,年龄:18,☏:110
1 学号:stu0003,姓名:zhangsan,性别:female,年龄:12,☏:123
请输入需要删除的学生的标号(0-1),其它则返回1
删除成功！
    ❀❀欢迎mkh:
    
      ❤1.添加学生
      ❤2.查看学生
     